In [1]:
import pandas as pd
import numpy as np
import ast
from functools import reduce
from tabulate import tabulate

custom_diff = lambda z: reduce(lambda x, y: abs(x - y), z)

In [2]:
def consecutive(x):
    if len(x) == 2:
        # если два, то вырождается
        return 1 if x[-1] - x[0] == 1 else 0
    if len(x) > 2:
        return 1 if (x[0] < x[-1]) and (len(x)*(x[0]+x[-1])/2 == sum(x)) else 0

In [47]:
sents.iloc[4693].tokens

"{'среди': [91115], 'офицеров': [91116], 'чью': [91117], 'карьеру': [91118], 'он': [91119], 'продвигал': [91120], 'был': [91121], 'и': [91122], 'молодой': [91123], 'иван': [91124], 'саввич': [91125], 'горголи': [91126], 'этого': [91127], 'грека': [91128], 'на': [91129], 'русской': [91130], 'службе': [91131], 'называли': [91132], 'одним': [91133], 'их': [91134], 'самых': [91135], 'красивых': [91136], 'людей': [91137], 'россии': [91138]}"

In [46]:
sents.iloc[4693].poss

"{'ADP': [91115, 91129], 'NOUN': [91116, 91118, 91124, 91125, 91126, 91128, 91131, 91137, 91138], 'DET': [91117, 91127, 91134], 'PRON': [91119], 'VERB': [91120, 91121, 91132], 'CONJ': [91122], 'ADJ': [91123, 91130, 91135, 91136], 'NUM': [91133]}"

In [48]:
query = '"был" молодой+ADJ NOUN'

search(query)

NameError: name 'search' is not defined

In [61]:
consecutive([2, 1])

0

In [326]:
def search(request, df=sents):
    sub_req = request.split()

    el_chars = {}
    for element in sub_req:
        with_pos = False
        if element[0] == '"':
            mode = "lemmas"
        elif element[0].isascii():
            mode = "poss"
        else:
            mode = "tokens"
        if "+" in element:
            with_pos = True
        print(element, mode)
        el_chars.update({element: word_search(element=element,
                                              sent_df=df, mode=mode,
                                              with_pos=with_pos)})
        
    if len(el_chars) == 1:
        sent, word_indices = list(el_chars[request].keys()), list(el_chars[request].values())
        print_out(sent, word_indices)

    else:
        common_sents = list(reduce(lambda x, y: x & y.keys(), el_chars.values()))

        for sent in common_sents:
            # беру разницу элементов массива, она должна быть равна длине массива -1
            listik = []
            for key in el_chars:
                listik.append(el_chars[key][sent])
            grid = np.array(np.meshgrid(*listik)).T.reshape(-1, len(el_chars))
            print(grid)
            diffs = np.apply_along_axis(consecutive, arr=grid, axis=1)
            print(diffs)
            word_indices = diffs[np.where(diffs == 1)]
            if word_indices.size > 0:
                print_out(sent, word_indices)

In [222]:
def print_out(sent_indice, word_indice, df=meta):
    if sent_indice:
        row = df.iloc[sent_indice].reset_index()
        np.apply_along_axis(lambda a: print(a), arr=row, axis=1)
    #print(row.sentence)
    #print(row.title)
    #print(row.date)
    print("")

In [322]:
def word_search(element, sent_df=sents, word_df=words, mode="tokens", with_pos=False):
    
    if mode == "lemmas":
        element = element[1:-1]

    if with_pos:
        word, pos = element.split("+")
        word_indices = word_search(word, sent_df, word_df, mode)
        pos_indices = word_search(pos, sent_df, word_df, mode="poss")
        indices = word_indices.keys() & pos_indices.keys()
        morph_indices = []
        for sent in indices:
            morph_indices.append(list(set(word_indices[sent]) & set(pos_indices[sent])))

    else:
        v = np.vectorize(lambda x: element in x)
        indices = np.where(v(sent_df[mode]) == True)[0]
        row = sent_df.iloc[indices]
        morph_indices = row[mode].apply(lambda x: x[element])
        if with_pos:
        #print(row[mode].apply(lambda x: x[word]))  # индексы находятся списками
            morph = word_df.iloc[morph_indices.sum()]
            u = np.vectorize(lambda x: x == pos)

    if not any(morph_indices):
        indices = []

    return dict(zip(indices, morph_indices))

In [55]:
meta = pd.read_csv("sents_meta.csv")
sents = pd.read_csv("parsed_sents (2).csv")
sents = sents.applymap(lambda x: ast.literal_eval(x))
words = pd.read_csv("morph_info.csv")

In [56]:
sents.iloc[5346].lemmas

{'прививать': [105356],
 'апельсин': [105357],
 'к': [105358],
 'ель': [105359],
 'и': [105360],
 'надеяться': [105361],
 'на': [105362],
 'добрый': [105363],
 'плод': [105364]}

In [161]:
meta.to_csv("sents_meta.csv", index=0)
sents.to_csv("whyyyyyyy.csv", index=0)
words.to_csv("morph_info.csv", index=0)

In [ ]:
word_search('знать+NOUN', sent_df=sents, word_df=words, mode="tokens", with_pos=True)

In [ ]:
https://stackoverflow.com/questions/24893977/whats-the-best-way-to-regex-replace-a-string-in-python-but-keep-its-case

In [26]:
import re
def replace_keep_case(word, replacement, text):
    def func(match):
        g = match.group()
        if g.islower(): return replacement.lower()
        if g.istitle(): return replacement.title()
        if g.isupper(): return replacement.upper()
        return replacement      
    return re.sub(word, func, text, flags=re.I)

In [6]:
text = "Eggs with eggs, bacon and spam are good for breakfast... EGGS!"
replace_keep_case("eggs", "spam", text)

'Spam with spam, bacon and spam are good for breakfast... SPAM!'

In [45]:
element = "да"

v = np.vectorize(lambda x: element in x)
indices = np.where(v(sents["tokens"]) == True)[0]

In [77]:
sents.iloc[2131].tokens

{'богачи': [44028],
 'и': [44029, 44038, 44043],
 'знать': [44030],
 'вообще': [44031],
 'сваливали': [44032],
 'бремя': [44033],
 'войны': [44034],
 'на': [44035],
 'пролетария': [44036],
 'как': [44037],
 'везде': [44039],
 'причем': [44040],
 'обогащались': [44041],
 'поставщики': [44042],
 'банкиры': [44044]}

In [78]:
sents.iloc[2131].poss["NOUN"] = [44028, 44030, 44033, 44034, 44036, 44042, 44044]
sents.iloc[2131].poss["VERB"] = [44032, 44041]

In [82]:
words.iloc[44028].tag

'Case=Nom|Gender=Masc|Number=Plur'

In [83]:
words.iloc[44030].pos = "NOUN"
words.iloc[44030].tag = "Case=Nom|Gender=Fem|Number=Sing"

In [ ]:
sents.ro_csv(")

In [47]:
consecutive([441796, 441799])

1

In [50]:
(441796+441799)

883595

In [12]:
%%time

query = "ты был"

results = search(query, sents, words, meta)

ты tokens
был tokens
CPU times: total: 31.2 ms
Wall time: 18 ms


In [5]:
query = "вот как"

results = search(query, sents, words, meta)

вот tokens
как tokens


In [22]:
words.iloc[1]

token                                             простите
lemma                                             простить
pos                                                   VERB
tag      Mood=Imp|Number=Plur|Person=2|VerbForm=Fin|Voi...
Name: 1, dtype: object

In [38]:
words[["token", "tag"]].values

array([['начну',
        'Mood=Ind|Number=Sing|Person=1|Tense=Notpast|VerbForm=Fin|Voice=Act'],
       ['простите',
        'Mood=Imp|Number=Plur|Person=2|VerbForm=Fin|Voice=Act'],
       ['с', '_'],
       ...,
       ['эпиграфа', 'Case=Gen|Gender=Masc|Number=Sing'],
       ['их', '_'],
       ['смысле', 'Case=Loc|Gender=Masc|Number=Sing']], dtype=object)

In [117]:
i = 0
def get_range(obj):
    if obj:
        return range(min(obj)[0], max(obj)[0]+1)
    else:
        pass

In [180]:
i = 0
def get_capitals(obj):
    if obj:
        return int(min(obj)[0])
    else:
        pass

In [88]:
min(sents.tokens[0].values())[0]

0

In [194]:
capitals = sents["tokens"].apply(lambda x: get_capitals(x.values()))

In [201]:
capitals = capitals.dropna().astype(int)

In [230]:
capitalized = words.iloc[capitals]
capitalized = capitalized.token.apply(lambda x: x.capitalize()).rename("caps")

In [235]:
words["caps"] = pd.concat([words.drop(capitals).token, capitalized])

In [219]:
capitalized = capitalized.token.apply(lambda x: x.capitalize()).rename("caps")

In [ ]:
capitalized = capitalized.token.apply(lambda x: x.capitalize()).remame("caps")

In [314]:
a = sents["tokens"].apply(lambda x: get_range(x.values())).rename("ranges")
sents_ranged = pd.concat([sents, a], axis=1)

In [315]:
da = lambda x: f'<div class="tooltip2">{x[0]}<span class="tooltiptext2">{x[1]}</span></div>'

tooltips = np.apply_along_axis(da, arr=words[["caps", "tag"]].values, axis=1)

In [316]:
tooltipped_words = pd.concat([words, pd.DataFrame({"tooltips": tooltips})], axis=1)

In [317]:
def ramble_tips(tokens_range, meta_tooltips=tooltipped_words.tooltips):
    if tokens_range:
        return " ".join(meta_tooltips[token_idx] for token_idx in tokens_range)
    else:
        pass

In [318]:
meta["tooltipped"] = sents_ranged.ranges.apply(ramble_tips).rename("tooltipped")

In [ ]:
meta.tooltipped[2]

In [309]:
sents_tooltipped = pd.concat([meta, sents_ranged.ranges.apply(ramble_tips).rename("tooltipped")], axis=1)

In [320]:
meta.to_csv("meta_tooltipped.csv", index=0)

In [295]:
tooltipped_words.iloc[:, [0, -1]].to_csv("tooltips.csv", index=0)

In [296]:
from markupsafe import Markup
value = Markup('<strong>The HTML String</strong>')

In [292]:
tooltipped_words.to_csv("tooltipped_words.csv", index=0)

In [167]:
tooltip_wrap(1, word_df=words, meta_df=meta)

'<div class="tooltip">простите<span class="tooltiptext">Mood=Imp|Number=Plur|Person=2|VerbForm=Fin|Voice=Act</span></div>"'

In [19]:
len(list(zip(results[0::2], results[1::2]))[::2])

8

In [41]:
a = [[0], [1], [2], [3]]
flat_list = [item for sublist in a for item in sublist]

In [51]:
def flatten(x):
    l = list(x.values())
    return [item for sublist in l for item in sublist]

In [66]:
sent_dict = sents.lemmas.apply(flatten).explode().drop_duplicates()

In [72]:
new_words = pd.concat([words, pd.Series(list(sent_dict.index))], axis=1)

In [ ]:
word_search("NOUN", mode="poss", sent_df=sents, word_df=words)

In [143]:
%%timeit

element = "я"

v = np.vectorize(lambda x: element in x)
indices = np.where(v(sents["tokens"]) == True)[0]
row = sents.iloc[indices]
morph_indices = row["tokens"].apply(lambda x: x[element])

7.48 ms ± 112 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [155]:
%%timeit

temp = new_words[new_words["token"] == element]
morph_indices1 = list(temp.index)
indices1 = temp[0].values

21.2 ms ± 196 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [159]:
(indices == indices1).all()

True

In [167]:
import numpy as np
import re
from functools import reduce
import pymorphy2

morph = pymorphy2.MorphAnalyzer()


def is_ordered(nums):
    if np.array_equal(np.sort(nums), nums):
        return True
    else:
        return False


def consecutive(l):
    setl = set(l)
    M, m = max(l), min(l)
    return len(l) == len(setl) and setl == set(range(m, M+1)) and is_ordered(l)
 

    
def search(request, sent_df, word_df, meta_df):
    sub_req = request.split()
    info = []

    el_chars = {}
    for element in sub_req:
        with_pos = False
        if element[0] == '"':
            mode = "tokens"
        elif element[0].isascii():
            mode = "poss"
        else:
            mode = "lemmas"
        if "+" in element:
            with_pos = True
        print(element, mode)
        el_chars.update({element: word_search(element=element,
                                              sent_df=sent_df,
                                              word_df=word_df,
                                              mode=mode,
                                              with_pos=with_pos)})
        
    if len(el_chars) == 1:
        sents, word_indices = list(el_chars[request].keys()), list(el_chars[request].values())
        for sent in sents:
            new_info = extract_info(sent, word_indices, meta_df, word_df, 1)
            if new_info:
                info.append(new_info)

    else:
        common_sents = list(reduce(lambda x, y: x & y.keys(), el_chars.values()))

        for sent in common_sents:
            listik = []
            for key in el_chars:
                listik.append(el_chars[key][sent])
            grid = np.array(np.meshgrid(*listik)).T.reshape(-1, len(el_chars))
            diffs = np.apply_along_axis(consecutive, arr=grid, axis=1)
            word_indices = diffs[np.where(diffs == 1)]
            if word_indices.size > 0:
                info.append(extract_info(sent, word_indices, meta_df, word_df))

    return info


def replace_keep_case(word, replacement, text):
    def func(match):
        g = match.group()
        if g.islower(): return replacement.lower()
        if g.istitle(): return replacement.title()
        if g.isupper(): return replacement.upper()
        return replacement
    return re.sub(rf"\b{word}\b", func, text, flags=re.I)


def highlight(sentence, string):
    sent_check = sentence.lower()
    
    if not string[0].isascii() and f"<mark>{string}</mark>".lower() not in sent_check:
        return replace_keep_case(string, f'<mark>{string}</mark>', sentence)
    else:
        return sentence


def highlight2(sentence, string):
    red = "\033[31m"
    nul = "\033[0m"
    sentence = re.sub(
        r"\b({})\b".format(re.escape(string)),
        r"{}\1{}".format(red, nul),
        sentence,
        flags=re.I,
    )
    return sentence


def extract_info(sent_indice, word_indice, meta_df, word_df, num=2):
    if sent_indice:
        row = meta_df.iloc[sent_indice]
        sent = row.sentence
        if num == 1:
            words = [word_df.iloc[item].token for sublist in word_indice for item in sublist]
            for word in words:
                sent = highlight(sent, word)
        return {"index": str(sent_indice), "sents": sent,
               "titles": row.title, "dates": row.date}
    else:
        return None
    
    
def word_search(element, sent_df, word_df, mode="tokens", with_pos=False, is_normal=False):
    
    res = {}
    
    #print(element, mode, with_pos)

    if mode == "lemmas" and not is_normal:
        elements = query_parse(element)
        for el in elements:
            res.update(word_search(el, sent_df, word_df, mode, with_pos, is_normal=True))
        return res

    elif mode == "tokens" and not with_pos:
        element = element[1:-1]

    if with_pos:
        indices = []
        word, pos = element.split("+")
        print(word, pos)
        word_indices = word_search(word, sent_df, word_df, mode)
        pos_indices = word_search(pos.upper(), sent_df, word_df, mode="poss")
        indices_candidates = word_indices.keys() & pos_indices.keys()
        morph_indices = []
        for sent in indices_candidates:
            candidate = list(set(word_indices[sent]) & set(pos_indices[sent]))
            if candidate:
                indices.append(sent)
                morph_indices.append(candidate)

    else:
        v = np.vectorize(lambda x: element in x)
        indices = np.where(v(sent_df[mode]) == True)[0]
        row = sent_df.iloc[indices]
        morph_indices = row[mode].apply(lambda x: x[element])

    if not any(morph_indices):
        indices = []

    return dict(zip(indices, morph_indices))


def query_parse(word):
    variants = []

    for i in morph.parse(word):
        form = i.normal_form
        if form not in variants:
            variants.append(form)

    return variants


def morphy_converter(x):
    converter = {"ADJF": "ADJ", "ADJS": "ADJ", "COMP": "ADJ",
                "INFN": "VERB", "PRTF": "VERB", "PRTS": "VERB", "GRND": "VERB",
                "NUMR": "NUM", "ADVB": "ADV", "PREP": "ADP", "PTCL": "PART"}
    return converter[x] if x in converter else x

In [58]:
def highlight(sentence, string):
    # не работает, если слово есть в предложении, но без чего-то
    words = " ".join(x for x in string.split("+")).split()
    string_length = len(words)
    
    sent_words = sentence.split()
    sent_idx = sent_words.index(words[0])
    words_to_change = " ".join(sent_words[sent_idx+x] for x in range(string_length))
    
    print(words_to_change)
    
    return replace_keep_case(words_to_change, f'<mark>{words_to_change}</mark>', sentence)

In [47]:
res = word_search("ели", sents, words, mode="lemmas")

ели lemmas False
есть lemmas False
ель lemmas False


In [8]:
word_search("ель", sents, words, mode="lemmas")

{5346: [105359], 26041: [476152]}

In [ ]:
word_search("есть", sents, words, mode="lemmas")

In [29]:
y = {"да" : 1}
x = {"нет": 2}
z = {"ах": 2}
a = {"sх": 3}

x.update(y)

In [69]:
a = [[91121, 91123, 91116],
 [91121, 91123, 91118],
 [91121, 91123, 91124],
 [91121, 91123, 91125],
 [91121, 91123, 91126],
 [91121, 91123, 91128],
 [91121, 91123, 91131],
 [91121, 91123, 91137],
 [91121, 91123, 91138]]

In [78]:
def check_is_consecutive(l):
    maximum = max(l)
    if sum(l) == maximum * (maximum+1) /2 : 
         return True
    return False

In [ ]:
def consecutive(l):
    setl = set(l)
    M, m = max(l), min(l)
    return len(l) == len(setl) and setl == set(range(m, M+1)) and is_ordered(l)

In [136]:
consecutive([91121, 91123, 91125])

False

In [71]:
np.apply_along_axis(consecutive, arr=a, axis=1)

array([0, 0, 0, 1, 0, 0, 0, 0, 0])

In [72]:
consecutive([91121, 91123, 91125])

True

In [168]:
query = '"молодой"+ADJ'

results = search(query, sents, words, meta)

"молодой"+ADJ tokens
"молодой" ADJ


In [163]:
query.split("+")

['"молодой"', 'ADJ']

In [170]:
results

[{'index': '20995',
  'sents': 'Итак мы видим, что, по мнению Александра, в 1801г. для успешной борьбы с <mark>молодой</mark> Францией «момент еще не настал».',
  'titles': 'Правда о войне 1812 года',
  'dates': '2004'},
 {'index': '4229',
  'sents': '<Mark>Молодой</Mark> литератор Джек Алтаузен (Я.М. Алтаузен: 1907-1942) писал: Я предлагаю Минина расплавить, Пожарского.',
  'titles': 'Первая научная история войны 1812 года',
  'dates': '2018'},
 {'index': '12935',
  'sents': '<Mark>Молодой</Mark> человек, обливаясь кровью, упал. ...Граф Ростопчин со своей свитой не успел покинуть двор, как опьяненная первой пролитой кровью толпа бросилась на израненного Михаила Верещагина.',
  'titles': 'Первая научная история войны 1812 года',
  'dates': '2018'},
 {'index': '16267',
  'sents': '229 Да, среди них было много конскриптов, юных и неопытных новобранцев, которых прозвали «Марии-Луизы» (в честь <mark>молодой</mark> и привлекательной жены Наполеона), но любой мало-мальски разбирающийся в вое

In [8]:
word_search("ель", sent_df, word_df, mode)

NameError: name 'sent_df' is not defined

In [16]:
def highlight(sentence, string):
    print(string[0])
    if string[0].isascii():
        
    else:
        return replace_keep_case(string, f'<mark>{string}</mark>', sentence)

In [58]:
import spacy
from spacy import displacy

In [6]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [5]:
def query_parse(word):
    variants = []

    for i in morph.parse(word):
        form = i.normal_form
        if form not in variants:
            variants.append(form)

    return variants


def morphy_converter(x):
    converter = {"ADJF": "ADJ", "ADJS": "ADJ", "COMP": "ADJ",
                "INFN": "VERB", "PRTF": "VERB", "PRTS": "VERB", "GRND": "VERB",
                "NUMR": "NUM", "ADVB": "ADV", "PREP": "ADP", "PTCL": "PART"}
    return converter[x] if x in converter else x

In [10]:
query_parse("ели")

['есть', 'ель']

In [4]:
def morphy_converter(x):
    converter = {"ADJF": "ADJ", "ADJS": "ADJ", "COMP": "ADJ",
                "INFN": "VERB", "PRTF": "VERB", "PRTS": "VERB", "GRND": "VERB",
                "NUMR": "NUM", "ADVB": "ADV", "PREP": "ADP", "PTCL": "PART"}
    return converter[x] if x in converter else x

In [55]:
predictor.predict(["ели"])[0].normal_form

1/1 [==============================] - 0s 18ms/step


[<normal_form=есть; word=ели; pos=VERB; tag=Mood=Ind|Number=Plur|Tense=Past|VerbForm=Fin|Voice=Act; score=0.9919>]

In [3]:
from rnnmorph.predictor import RNNMorphPredictor

predictor = RNNMorphPredictor(language="ru")

In [134]:
words = words.drop(list(words[(words.token == "е") | (words.token == "п")].index), axis=0)

In [150]:
sents.iloc[45].tokens

{'и': [1058],
 'как': [1059],
 'только': [1060],
 'появился': [1061],
 'реальный': [1062],
 'молодой': [1063],
 'парень': [1064],
 'с': [1065],
 'будущим': [1066],
 'они': [1067],
 'всем': [1068],
 'миром': [1069],
 'стали': [1070],
 'его': [1071],
 'топить': [1072]}

In [151]:
sents.iloc[45].poss

{'CONJ': [1058],
 'ADV': [1059, 1060],
 'VERB': [1061, 1070, 1072],
 'ADJ': [1062, 1063],
 'NOUN': [1064, 1066, 1069],
 'ADP': [1065],
 'PRON': [1067, 1071],
 'DET': [1068]}

In [139]:
words = words.reset_index(drop=True)

In [101]:
def highlight(sentence, string):
    if string[0] == '"':
        string = string[1:-1]
    if string[0].isascii():
        return sentence
    else:
        return replace_keep_case(string, f'<mark>{string}</mark>', sentence)

In [103]:
highlight("Я хочу пиццы!", '"хочу"')

"
"хочу"


'Я <mark>хочу</mark> пиццы!'

In [102]:
a = '"хочу"'
a[1:-1]

'хочу'

In [ ]:
я был
хайлайты

In [7]:
words = words.token

In [13]:
words = pd.DataFrame(words)

In [24]:
meta.iloc[6586].sentence

'Статья VII Ввиду неизвестности, в коей обе Высокие Договаривающиеся Стороны поныне еще находятся в рассуждении будущих намерений французского правительства, они оставляют за собой право, сверх постановленного выше сего, условиться, смотря по стечению обстоятельств, и о тех различных случаях, вследствие коих нужно будет также употребить взаимные их силы.'

In [3]:
import csv

In [26]:
sents = pd.read_csv("parsed_sents.csv")

In [ ]:
sents.to_csv("parsed_sents.csv", index=0, quoting=csv.QUOTE_NONE)

In [41]:
meta = pd.read_csv("sents_meta.csv", engine='python', on_bad_lines='skip')
#sents = pd.read_csv("parsed_sents.csv", on_bad_lines='skip')
#sents = sents.applymap(lambda x: ast.literal_eval(x))
#words = pd.read_csv("morph_info.csv", on_bad_lines='skip')

In [42]:
meta.isna().sum()

url         0
title       0
date        0
sentence    0
dtype: int64